In [1]:
#driver.find_element(By.?) ? can be CSS_SELECTOR,ID etc
#driver.find_elements(By.?) to find multiple elements with the selector or ids. we will get array of elements.
#element.text.strip() to get text of element. 
#element.get_attribute('href') to get the url in the element.

In [13]:
#this is for scrape the articles of volumnes and issues.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import csv
import time
#intialize the chrome webdriver
driver=webdriver.Chrome()
driver.get('https://www.tandfonline.com/toc/tjis20/33/5?nav=tocList')
# Wait for the page to fully load
driver.implicitly_wait(20)
def getVolumeIssues(url):
    driver=webdriver.Chrome()
    driver.implicitly_wait(20)
    try:
        if not url:
            return 0
        driver.get(url)
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        
        
        vol_title=driver.find_element(By.CSS_SELECTOR,'div.toc-title')
        title=vol_title.text.strip()
    
       
    
        articles=driver.find_elements(By.CSS_SELECTOR,'div.art_title')
        for article in articles:
            try:
               # Wait for the element to be present before trying to access it
                url_ele = WebDriverWait(article, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'a.ref'))
                )
                # Get the URL
                article_url = url_ele.get_attribute('href')
                article_title = article.find_element(By.CSS_SELECTOR,'span.hlFld-Title')
                title_text = article_title.text.strip()
            except Exception as e:
                print("error at article url",e)
                break
            with open('european_journal_of_information_systems.csv', mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow([title, article_url, title_text])
                    file.flush() 
    except Exception as e:
        print("An error occurred while processing volume issues:", e)
    driver.quit()
   
    
def findIssues(url):
     #find the issues
    driver=webdriver.Chrome()
    driver.get(url)
    print(url)
    driver.implicitly_wait(20)
    try:
        issue_ele = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.loi-issues-scroller'))
            )
        issue_links = issue_ele.find_elements(By.CSS_SELECTOR, 'a')
        for issue in issue_links:
            href = issue.get_attribute('href')
            getVolumeIssues(href)
    except Exception as e:
        print(f"Error while finding issues for {url}: {e}")
        getVolumeIssues(url)
    driver.quit()
urls=['https://www.tandfonline.com/toc/tjis20/3/4?nav=tocList']
for i in urls:
    findIssues(i)
# years_ele=driver.find_element(By.CSS_SELECTOR,'div.yearSliderInner')
# years=years_ele.find_elements(By.CSS_SELECTOR,'a.expander')
# for year in years:
#     url=year.get_attribute('href')
#     findIssues(url)


driver.quit()

https://www.tandfonline.com/toc/tjis20/3/4?nav=tocList
Error while finding issues for https://www.tandfonline.com/toc/tjis20/3/4?nav=tocList: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF747B238A5+3004357]
	(No symbol) [0x00007FF7477B9970]
	(No symbol) [0x00007FF74766582A]
	(No symbol) [0x00007FF7476B5B8E]
	(No symbol) [0x00007FF7476B5E7C]
	(No symbol) [0x00007FF7476FEC27]
	(No symbol) [0x00007FF7476DBC1F]
	(No symbol) [0x00007FF7476FBA4C]
	(No symbol) [0x00007FF7476DB983]
	(No symbol) [0x00007FF7476A7628]
	(No symbol) [0x00007FF7476A8791]
	GetHandleVerifier [0x00007FF747B4A00D+3161901]
	GetHandleVerifier [0x00007FF747B9E060+3506048]
	GetHandleVerifier [0x00007FF747B9400D+3465005]
	GetHandleVerifier [0x00007FF747910EEB+830987]
	(No symbol) [0x00007FF7477C467F]
	(No symbol) [0x00007FF7477C09D4]
	(No symbol) [0x00007FF7477C0B6D]
	(No symbol) [0x00007FF7477B0149]
	BaseThreadInitThunk [0x00007FFDE9767344+20]
	RtlUserThreadStart [0x00007FFDE9AC26B1+33]

error at article url Message: 


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

import csv
import time
euro_data=pd.read_excel('EJIS1.xlsx')
def getAuthorsData(authors):
    auth_data=[]
    name,url,desc,email='','','',''
    for auth in authors:
        hover = ActionChains(driver).move_to_element(auth)
        hover.perform()
        try:
            auth_details=auth.find_element(By.CSS_SELECTOR,'a.author')
            name=auth_details.text.strip()
           
            url=auth_details.get_attribute('href')
        except Exception as e:
            print('error at name and email of author',e)
        try:
            # Wait for the overlay description to appear
            desc = auth.find_element(By.CSS_SELECTOR,'span.overlay').text.strip()
        except Exception as e:
            print("error at description",e)
        try:
            email = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'span.corr-email a'))
            ).text.strip()
        except Exception as e:
            print("error at author email",e)
        auth_data.append([name,url,desc,email])
        ActionChains(driver).move_to_element(driver.find_element(By.TAG_NAME, 'body')).perform()
    return auth_data
        
with open('European_journals_articles_data.csv',mode='a',newline='',encoding='utf-8') as file:
    writer=csv.writer(file)
    writer.writerow(['Journal_Title',"URL", "Article_Title",'Volume_Issue','Month_Year','Abstract','Keywords','Author_name','Author_url','Author_Address','Author_email'])  # Write the header row
for index,row in euro_data.iterrows():
    final_data=[]
    driver=webdriver.Chrome()
    driver.get(row['URL'])
    driver.implicitly_wait(10)
    try:
        #to avoid no such element error use webdriverwait
        vol_year_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.issue-heading'))
        )
        vol_year = vol_year_element.text.strip()
        
        vol_issue_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.issue-heading a'))
        )
        vol_issue = vol_issue_element.text.strip()
    except Exception as e:
        print("Error occured",e)
    try:
        abstract=driver.find_element(By.CSS_SELECTOR,'div.hlFld-Abstract p.last').text.strip()
    except Exception as e:
        print("error at abstraction",e)
    try:
        keyword_list_element=driver.find_element(By.CSS_SELECTOR,'div.hlFld-KeywordText')
        keyword_list=keyword_list_element.find_elements(By.CSS_SELECTOR,'a.kwd-btn')
        keywordlist=[]
        for key in keyword_list:
            keyword=key.text.strip()
            keywordlist.append(keyword)
    except Exception as e:
        print(e)
    final_data=[row['Title'],row['URL'],row['Article_Title'],vol_issue,vol_year,abstract,keywordlist]
    try:
        authors=driver.find_elements(By.CSS_SELECTOR,'div.addAuthorInfo')
        # authors=driver.find_elements(By.CSS_SELECTOR,'div.entryAuthor')
        # authdata=getAuthorsData(authors)
        for auth in authors:
            try:
                auth_title=auth.find_element(By.CSS_SELECTOR,'h4').text.strip()
            except Exception as e:
                auth_title='NA'
            try:
                auth_email=auth.find_element(By.CSS_SELECTOR,'div.NLM_bio a').text.strip()
            except Exception as e:
                auth_email='NA'
            try:
                auth_desc=auth.find_element(By.CSS_SELECTOR,'div.NLM_bio p').text.strip()
            except Exception as e:
                auth_desc='NA'
            with open('European_journals_articles_data.csv', mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(final_data + [auth_title,auth_desc,auth_email])
                file.flush()
    except Exception as e:
            print(f"Error processing author data on {row['URL']}: {e}")
    # try:
    #     authors=driver.find_elements(By.CSS_SELECTOR,'div.entryAuthor')
    #     authdata=getAuthorsData(authors)
    #     for i in authdata:
    #         with open('european_journals4.csv', mode='a', newline='', encoding='utf-8') as file:
    #             writer = csv.writer(file)
    #             writer.writerow(final_data + i)
    #             file.flush()
    # except Exception as e:
    #         with open('european_journals4.csv', mode='a', newline='', encoding='utf-8') as file:
    #             writer = csv.writer(file)
    #             writer.writerow(final_data + ["N/A", "N/A", "N/A"])
    #             file.flush()
    #         print(f"Error processing author data on {row['URL']}: {e}")
   
    driver.quit()


    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
